# Predicting Future Movement Stock Using Momentum Statistics

The purpose of this is the development and testing of using 6 momentum based trading signals to predict next day momentum buy/sell/hold signals.

The inspiration for this is from this [article](https://www.sciencedirect.com/science/article/pii/S2405918815300179#tbl4).

### Summary of results

Initial testing with the trading signals used gave a return of 48.8% when trading on the SP500. Using a Keras model inspired by the one written in the article, results got up to 55% returns.

In [526]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import *
from keras.optimizers import Adam
from keras import initializers
import os
import csv
import pickle
import math
import random
import matplotlib.pyplot as plt
from sklearn import preprocessing

#standard adam optimizer for neural networks
adam = Adam(lr=0.0001, beta_1=0.5)

In [221]:
with open("sp500tickers.pickle", "rb") as f:
    tickers = pickle.load(f)
#adding in extra tickers that aren't in the sp500
#sp500tickers.pickle contains the sp500 scraped from wikipedia
tickers.append('CGC')
tickers.append('CRON')
tickers.append('TLRY')
tickers.append('VMW')
tickers.append('CRON')
tickers.append('CRM')
tickers.append('CMI')
tickers.append('BOX')
tickers.append('SSTK')

### The Six Input Values

$MA_{15}$, $MACD_{26}$, $K_{14}$, $D_{3}$, $RSI_{14}$, $R_{14}$ are the aforementioned momentum based trading signals. 
$MA_{15}$ is given as
$$MA_{15} = \frac{1}{15}\sum_{i=1}^{15}CP(i)$$ where $CP(i)$ is the closing price on the given day.

$MACD_{26}$ is given as
$$MACD_{26} = EMA_{12}-EMA_{26}$$ 
where $EMA_{i}$ is given as
$$EMA_{i} = (CP(i) - EMA_{i-1})\times \frac{2}{num. days considered} + EMA_{i-1}$$
where number of days considered is either 12 or 26.


$K_{14}$ is given as
$$K_{14} = 100\times \frac{CP(i) - L_{14}}{H_{14}-L_{14}} $$
where $L_{14}$ is the lowest closing price in the last 14 days and $H_{14}$ is the highest closing price in the last 14 days. 


$D_{3}$ is simply the $MA_{3}$ of $K_{14}$


$RSI_{14}$ is given as
$$RSI_{14} = 100 - \frac{100}{1+RS}, RS = \frac{average 14 days up closes}{average 14 days down closes} $$


$R_{14}$ is given as
$$R_{14} = \frac{H_{14}-CP(i)}{H_{14}-L_{14}}$$
where $H_{14}$ and $L_{14}$ are the same as the values for $K$

Afterwards these 6 signals are combined with another trend indicator which is given as
* UP: if the closing price is greater than the $MA_{15}$ and the $MA_{15}$ has gone up over the last 5 days
* DOWN: if the closing price is less than the $MA_{15}$ and the $MA_{15}$ has gone down over the last 5 days
* NO TREND: otherwise

We will combine this with our final indicator to determine whether we should buy, sell, or hold in a market given the previous 6 signals. This final indicator, the output trend, is given as 
$$TR_{i} = \frac{CP(i)-min(cp)}{max(cp)-min(cp)}$$
where $max(CP)$ and $min(CP)$ are
$$max (CP) = max(CP_{i}, CP_{i+1}, CP_{i+2)$$
$$min (CP) = min(CP_{i}, CP_{i+1}, CP_{i+2)$$

Notice here that the output trend relies on future data. The model will use our 6 momentum input data to predict this output trend value.

In [487]:
stocks = pd.read_csv('sp500_joined_closes.csv')
#load sp500 data scraped from robinhood and joined into csv with all the close_prices

#initialize dataframes to calculate all statistics
stocks15 = pd.DataFrame()
macd = pd.DataFrame()
Ks = pd.DataFrame()
D = pd.DataFrame()
RSI = pd.DataFrame()
R = pd.DataFrame()
#trends are calculated by if current price > 15MA, and MA has risen over last five days then UP.
#if current price < 15MA, and MA has gone down in the last five days then DOWN
#otherwise NO trend
allTrends = pd.DataFrame()
#output trend is calculated 0.5 + (current price - lowest price in next 2 days)/(max price in next 2 days - lowest price in next 2 days)
#if trend is up, then output trend + 0.5 <- to boost signal to buy
outputTrend = pd.DataFrame()

#if stock data cannot be ready for any reason
errors = []

#small group of stocks to test on
ticks=['AMZN','AMD','AAPL','MSFT','NVDA', 'VMW','CRM','INTC']
counter = 0
#full sp500 stocks if tickers instead of ticks
for i in tickers:
    try:
        #reading in data per stock
        df = pd.read_csv('robinhood_stocks/{}.csv'.format(i))
        #setting dataframe index by date
        df.set_index('begins_at', inplace=True)

        stocks15[i] = df['close_price'].rolling(15).mean()
        macd[i] = df['close_price'].ewm(span=12).mean() - df['close_price'].ewm(span=26).mean()
        Ks[i] = 100*(df['close_price'] - df['close_price'].rolling(14).min())/(df['close_price'].rolling(14).max() - df['close_price'].rolling(14).min())
        D[i] = Ks[i].rolling(3).mean()
        R[i] = 100*((df['close_price'].rolling(14).max() - df['close_price'])/(df['close_price'].rolling(14).max() - df['close_price'].rolling(14).min()))
        outputTrend[i] = 0.5*((df['close_price'] - df['close_price'][::-1].rolling(3).min()[::-1]) / (df['close_price'][::-1].rolling(3).max()[::-1] - df['close_price'][::-1].rolling(3).min()[::-1]))
        trends = []
        # diff temporarily storing difference by day to find gain/loss
        diff = df['close_price'].rolling(2).apply(lambda x: x[1] - x[0])
        for j in range(len(outputTrend[i])):
            if stocks15[i][j] < df['close_price'][j]:
                if stocks15[i][j-4] < stocks15[i][j]:
                    outputTrend[i][j] += 0.5
                    trends.append(1)
                else:
                    trends.append(0)
            if stocks15[i][j] > df['close_price'][j]:
                if stocks15[i][j-4] > stocks15[i][j]:
                    trends.append(-1)
                else:
                    trends.append(0)

        gains = []
        losses = []
        for j in diff:
            if j > 0:
                gains.append(j)
                losses.append(0)
            elif j < 0:
                gains.append(0)
                losses.append(-j)
            else:
                gains.append(0)
                losses.append(0)

        gains = pd.DataFrame(np.array(gains))
        losses = pd.DataFrame(np.array(losses))
        RSI[i] = 100 - 100/(1+(gains[0].rolling(14).mean()/losses[0].rolling(14).mean()))
        allTrends[i] = np.array(trends)
        counter += 1
        if counter % 10 == 0:
            print(counter)
    except:
        print("error",i)
        errors.append(i)
        continue
        
#freeing any errors, if any
for i in errors:
    try:
        stocks15 = stocks15.drop(i, axis = 1)
    except:
        print('wasnt here')
    try:
        macd = macd.drop(i, axis = 1)
    except:
        print('wasnt here')
    try:
        Ks = Ks.drop(i, axis = 1)
    except:
        print('wasnt here')
    try:
        D = D.drop(i, axis = 1)
    except:
        print('wasnt here')
    try:
        RSI = RSI.drop(i, axis = 1)
    except:
        print('wasnt here')
    try:
        R = R.drop(i, axis = 1)
    except:
        print('wasnt here')
    try:
        outputTrend = outputTrend.drop(i, axis = 1)
    except:
        print('wasnt here')
    try:
        allTrends = allTrends.drop(i, axis = 1)
    except:
        print('wasnt here')


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
error DRE
170
180
190
200
210
error GGP
220
230
240
250
260
270
280
290
300
310
320
error MDLZ
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
error XL
500
510
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here
wasnt here


### Testing Raw Data

Now that we have all our data ready, we will first test that this trading signal is an accurate and useful one on when to buy and sell. From the article, we saw that it was graphically able to pinpoint correct buy/sell/hold results to generate strong returns. We will test this out with the entire SP500 +/- a few stocks with a base capital of 4000.

In [527]:
#testing training runs
#cash is starting price, portfolio is empty
cash = 4000
portfolio = {}
# list of stocks
stocks_list = list(outputTrend)
#logger is to keep track of trading log
logger = {}
last_day = 0
# i running for number of trading days. since ma need a few days, we are skipping ahead 
for i in range(20, len(outputTrend)-2):
    logs = []
    buys = []
    sells = []
    hold = []
    
    temp = cash
    assets = ''
    for asset in portfolio:
        temp += portfolio[asset]*stocks[asset][i]
        assets = assets + asset + ' ' + str(portfolio[asset]*stocks[asset][i])+" "
    #printing assets in portfolio
    print(assets)
    #logging current portfolio
    logs.append(str(cash)+' asset values '+str(temp))
    # j for each stock
    for j in stocks_list:
        #if the trading signal is up and the trends are up, BUY
        if outputTrend[j][i] > 0.5 and allTrends[j][i] == 1:
            buys.append((j, outputTrend[j][i]))
        #if trading signal is down and trends are down, SELL
        elif outputTrend[j][i] < 0.5 and allTrends[j][i] == -1:
            sells.append(j)
        #otherwise HOLD
        else:
            hold.append(j)
    #sorting the buys by highest trading signal
    sorted_buys = sorted(buys, key = lambda x: x[1])[::-1]
    
    #if there are too many potential buys, we will only buy the top 20
    if len(sorted_buys) > 20:
        sorted_buys = sorted_buys[:20]
        
    #selling all sell stocks to make cap space to buy
    for sell in sells:
        if sell in portfolio:
            #SELL THE STOCK
            #cash goes up by how much it is now
            cash += portfolio[sell]*stocks[sell][i]
            logs.append('sold '+sell+' '+str(portfolio[sell])+' @'+str(stocks[sell][i]))
            del portfolio[sell]
    #all sells have been sold
    
    #evenly distributing the amount of cash to buy shares
    #could use some portfolio optimization here
    cash_per_stock = cash/len(sorted_buys)
    #number of stocks bought is buycount
    buyCount = 0
    
    for buy in sorted_buys:
        #if the stock price exceeds the amount allocated, then if we have enough money, buy anyways
        if stocks[buy[0]][i] >= cash_per_stock and cash-stocks[buy[0]][i] >= 0:
            #reduce cash by bought stock
            cash -= stocks[buy[0]][i]
            buyCount += 1
            #recalculate cash allocated per stock buy
            cash_per_stock = cash/(len(sorted_buys)-buyCount)
            #update portfolio with stock buys
            if buy[0] in portfolio:
                portfolio[buy[0]] += 1
            else:
                portfolio[buy[0]] = 1
            logs.append('buy '+buy[0]+' 1 @'+str(stocks[buy[0]][i]))
            
            #if stock price is less than allocated money, then see how many shares we can buy
        elif stocks[buy[0]][i] < cash_per_stock:
            num_shares = int(cash_per_stock/stocks[buy[0]][i])
            cash -= stocks[buy[0]][i] * num_shares
            logs.append('buy '+buy[0]+' '+str(num_shares)+' @'+str(stocks[buy[0]][i]))
            #bought however many < cash_per_stock
            #updating portfolio
            if buy[0] in portfolio:
                portfolio[buy[0]] += num_shares
            else:
                portfolio[buy[0]] = num_shares
    #log trading data
    logger[i] = logs
    last_day = i
    print(logs)

#printing final values
print('Final cash on hand:', cash)
for val in portfolio:
    cash += portfolio[val]*stocks[val][last_day]
print('Total asset val:',cash)


['4000 asset values 4000', 'buy SSTK 5 @34.82', 'buy BOX 9 @21.53', 'buy ZTS 3 @65.5382', 'buy YUM 2 @75.2048', 'buy WY 5 @34.3524', 'buy WAT 1 @188.14', 'buy DIS 2 @97.8205', 'buy VRSN 1 @108.94', 'buy URI 1 @144.4', 'buy TSN 2 @70.3533', 'buy FOX 7 @26.603', 'buy FOXA 7 @27.26', 'buy SYK 1 @149.1751', 'buy CRM 2 @98.9', 'buy RHI 3 @50.7666', 'buy PX 1 @140.9059', 'buy PYPL 2 @70.97', 'buy NOC 1 @290.4411', 'buy NWS 3 @13.9089', 'buy MSCI 1 @120.6207']
SSTK 172.85 BOX 185.4 ZTS 194.64690000000002 YUM 149.1852 WY 169.52249999999998 WAT 187.55 DIS 194.2632 VRSN 107.96 URI 141.48 TSN 140.6672 FOX 180.4621 FOXA 185.1997 SYK 148.6101 CRM 196.88 RHI 149.9115 PX 140.2168 PYPL 139.6 NOC 289.8989 NWS 41.4309 MSCI 120.3241 
['723.5659000000003 asset values 3959.6250000000005', 'buy ZTS 1 @64.8823', 'buy WLTW 1 @164.02', 'buy WDC 1 @86.4097', 'buy WMT 1 @86.6011', 'buy UDR 1 @37.5663', 'buy TSN 1 @70.3336', 'buy TRV 1 @131.4718', 'buy STX 1 @37.4602', 'buy NI 1 @26.0017', 'buy NWS 1 @13.8103']


['16.929900000000263 asset values 4201.6512999999995']
SSTK 201.25 BOX 221.20000000000002 ZTS 352.33400000000006 YUM 233.6946 WY 176.63049999999998 WAT 198.21 DIS 203.5926 VRSN 113.75 URI 150.09 TSN 230.02979999999997 FOX 211.47629999999998 FOXA 216.1474 CRM 215.16 RHI 161.883 PX 148.2888 PYPL 152.76 NOC 292.6593 NWS 79.9025 MSCI 125.5294 WMT 95.2173 UDR 38.1705 TRV 127.0422 STX 37.5269 NI 26.4299 XYL 65.854 VFC 69.1826 XEL 49.2302 RF 15.4616 
['16.929900000000263 asset values 4225.633299999999', 'sold UDR 1 @38.1705', 'buy FOX 1 @30.2109', 'buy NWS 1 @15.9805']
SSTK 202.75 BOX 227.5 ZTS 353.13 YUM 234.2595 WY 175.803 WAT 200.27 DIS 202.2344 VRSN 115.18 URI 151.5 TSN 232.9506 FOX 237.088 FOXA 212.74679999999998 CRM 214.74 RHI 161.883 PX 148.5152 PYPL 152.02 NOC 296.6916 NWS 94.4028 MSCI 126.1344 WMT 95.227 TRV 126.8955 STX 37.9362 NI 26.4397 XYL 66.4191 VFC 70.2964 XEL 48.94 RF 15.5203 
['8.909000000000265 asset values 4236.3825']
SSTK 208.25 BOX 230.9 ZTS 355.171 YUM 236.4597 WY 175.8

['1.1815000000002556 asset values 4311.740899999999', 'sold VRSN 1 @113.78', 'buy SSTK 1 @43.3', 'buy WRK 1 @62.9132']
SSTK 298.69 ZTS 428.5944 YUM 243.9225 DIS 329.06489999999997 URI 161.17 TSN 238.9878 FOX 271.3424 FOXA 242.0292 RHI 160.5771 PX 147.585 PYPL 148.52 NOC 303.5649 NWS 97.3626 WMT 95.3878 TRV 130.3425 STX 39.9259 XYL 66.1316 VFC 72.1271 RF 16.637 VIAB 58.2482 WYNN 162.9671 TWTR 22.58 FCX 16.0986 AES 10.2991 ZION 146.5068 WRK 122.4018 WBA 70.36 VZ 50.5218 UAL 63.13 UA 12.45 VLO 85.4897 
['8.748300000000256 asset values 4321.764100000001', 'sold TRV 1 @130.3425', 'sold TSN 3 @79.6626', 'buy OKE 1 @51.3736', 'buy NBL 1 @26.1805', 'buy ANTM 1 @225.0155', 'buy MOS 1 @24.8343', 'buy DVN 1 @37.787']
SSTK 301.0 ZTS 429.7884 YUM 244.6956 DIS 331.1481 URI 164.29 FOX 271.3424 FOXA 242.7929 RHI 163.6572 PX 148.0402 PYPL 151.3 NOC 307.086 NWS 96.7704 WMT 95.3682 STX 40.001999999999995 XYL 66.9942 VFC 73.1694 RF 16.8436 VIAB 59.2874 WYNN 163.9845 TWTR 22.23 FCX 16.8842 AES 10.4638 ZION

['3.0217000000002585 asset values 4594.37', 'sold FCX 1 @19.2905', 'sold RHI 3 @55.20399999999999', 'sold VZ 1 @50.326', 'buy QRVO 1 @70.7', 'buy HRS 1 @144.0519']
SSTK 320.03999999999996 ZTS 534.7685 YUM 332.2468 URI 183.67 FOX 323.4582 FOXA 254.3114 PX 160.1922 PYPL 167.68 NOC 312.9217 NWS 103.8732 WMT 102.714 STX 50.838 XYL 71.2972 VFC 78.4597 RF 18.3686 VIAB 65.4042 WYNN 177.4268 AES 11.3842 ZION 159.9312 WRK 135.3368 WBA 76.47 UAL 76.5 VLO 95.0471 OKE 56.809 NBL 31.3553 ANTM 247.7429 DVN 42.6098 M 26.1318 BOX 22.68 ZBH 122.9381 TRIP 35.45 QRVO 68.4 HRS 144.2002 
['23.4983000000002 asset values 4634.155200000001']
SSTK 321.37 ZTS 540.771 YUM 336.6076 URI 187.12 FOX 329.7024 FOXA 258.8908 PX 158.9453 PYPL 167.96 NOC 311.7051 NWS 103.8732 WMT 103.5586 STX 51.3597 XYL 72.0309 VFC 79.0792 RF 18.8507 VIAB 66.5414 WYNN 192.8247 AES 11.0257 ZION 161.2113 WRK 134.5932 WBA 77.4 UAL 76.87 VLO 96.6987 OKE 58.3286 NBL 32.5921 ANTM 248.1788 DVN 44.5589 M 26.364 BOX 22.71 ZBH 125.6462 TRIP 37.2 

/Users/harry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in double_scalars


...

['16.049200000000255 asset values 5389.36']
AES 51.7784 M 151.49200000000002 CRM 138.41 INTU 208.5685 PSA 211.9982 WY 147.8 UDR 146.5768 VRSK 109.9 OKE 67.78 APC 68.219 KR 25.761 SYY 133.0774 UA 107.35 UAA 116.05000000000001 ROST 84.9454 CTL 17.5844 HAS 181.4054 NDAQ 94.4637 VMW 451.38 FOX 88.7592 FOXA 177.9244 TWTR 91.6 TXT 67.3907 TTWO 121.51 SPGI 208.1656 TRIP 58.54 TGT 76.652 TSCO 222.4212 WELL 115.0 KIM 16.2182 NKE 75.6537 F 23.4306 MAT 17.68 WU 81.9452 CRON 14.04 ZTS 89.0692 VMC 131.7539 V 134.8983 UNH 255.1192 UAL 73.89 QCOM 59.3199 SO 43.8624 SCG 74.142 O 52.3479 PEG 52.0834 FE 34.3553 WMT 166.422 KMI 16.581 ADM 46.03 SLG 98.5146 NOV 41.4337 LB 35.718 
['16.049200000000255 asset values 5393.061', 'sold TXT 1 @67.3907', 'buy UDR 1 @36.6442', 'buy FOX 1 @44.3796']
AES 51.7784 M 153.3128 CRM 139.8 INTU 208.7681 PSA 212.6519 WY 148.48 UDR 180.792 VRSK 109.6 OKE 68.52 APC 69.3644 KR 25.8904 SYY 132.7988 UA 107.8 UAA 116.55 ROST 84.7559 CTL 17.6918 HAS 181.0676 NDAQ 93.7571 VMW 454.3

['1.2255000000002099 asset values 5455.3588', 'sold LB 1 @35.2861', 'buy CMCSA 1 @33.17']
AES 51.5408 CRM 137.84 INTU 204.3764 PSA 227.2725 UDR 185.10500000000002 VRSK 107.73 OKE 69.33 APC 73.1693 KR 28.568 SYY 136.36 ROST 84.9654 CTL 18.6383 HAS 185.1416 VMW 641.48 FOX 143.50650000000002 FOXA 193.6212 TTWO 118.15 SPGI 202.2303 TRIP 56.24 TGT 76.0368 TSCO 379.322 WELL 125.8 KIM 16.66 NKE 76.0926 ZTS 85.9438 V 131.2537 UNH 246.1495 UAL 70.16 QCOM 54.8307 SO 46.3996 SCG 160.4056 O 107.4634 PEG 53.0352 FE 36.0681 WMT 335.7868 KMI 17.4907 ADM 91.32 SLG 101.28 NOV 43.1018 WFC 55.1512 SSTK 48.79 WMB 26.728 VIAB 28.8171 DIS 103.2104 PFE 36.0383 CMCSA 33.09 
['3.3416000000002057 asset values 5455.0322000000015', 'sold VIAB 1 @28.8171', 'sold ZTS 1 @85.9438', 'buy AIZ 1 @104.9447']
AES 51.3824 CRM 139.77 INTU 204.6758 PSA 230.5116 UDR 186.4435 VRSK 109.26 OKE 70.23 APC 72.2928 KR 28.4685 SYY 138.72 ROST 84.9155 CTL 18.9115 HAS 189.4144 VMW 632.2 FOX 143.9247 FOXA 194.1788 TTWO 120.82 SPGI 204.3

AES 55.28 CRM 142.71 UDR 235.26 VRSK 114.58 KR 29.7725 SYY 139.44 ROST 89.485 CTL 18.248 HAS 198.42 VMW 605.52 TTWO 121.32 SPGI 198.0406 TGT 80.9783 TSCO 392.526 WELL 126.76 NKE 79.3146 V 139.5014 UNH 255.8966 UAL 81.83 QCOM 65.1369 SCG 168.1512 O 114.7862 WMT 445.7305 KMI 17.95 ADM 100.12 SLG 105.51 NOV 46.448 WFC 175.37009999999998 SSTK 52.77 WMB 61.9568 DIS 115.94 PFE 41.04 CMCSA 35.32 AIZ 105.4619 XEL 93.8204 WEC 65.9309 USB 52.93 TSS 93.4358 TMO 234.9262 HCP 26.26 ZBH 126.02 WM 89.3637 TLRY 24.84 YUM 81.59 
['12.254800000000131 asset values 5657.946400000002', 'sold AIZ 1 @105.4619', 'buy TLRY 1 @24.84', 'buy WBA 1 @67.42']
AES 54.52 CRM 144.44 UDR 234.89999999999998 VRSK 116.17 KR 29.8621 SYY 139.1 ROST 90.5426 CTL 18.1894 HAS 197.92 VMW 612.24 TTWO 122.21 SPGI 201.7315 TGT 81.6034 TSCO 398.50500000000005 WELL 127.66 NKE 80.3321 V 140.0706 UNH 256.7836 UAL 82.32 QCOM 64.8495 SCG 159.54 O 114.0688 WMT 446.22799999999995 KMI 17.98 ADM 100.5 SLG 105.82 NOV 46.5779 WFC 175.1616 SSTK 

['2.3904000000001133 asset values 5929.3862', 'sold SYY 3 @73.97', 'sold WFC 3 @58.46', 'buy TMO 1 @239.0234', 'buy FTI 1 @30.86', 'buy SYMC 1 @20.2', 'buy PG 1 @83.41', 'buy NLSN 1 @26.11']
CRM 152.68 UDR 279.78999999999996 VRSK 119.09 ROST 95.5611 CTL 21.36 HAS 198.62 VMW 613.04 TTWO 133.56 SPGI 207.05 TGT 87.5 TSCO 441.4 WELL 133.42 NKE 82.2 V 146.89 UNH 267.5572 UAL 87.42 QCOM 68.09 O 117.14 WMT 479.3 ADM 100.8 SSTK 165.12 PFE 41.52 CMCSA 36.99 XEL 143.0226 WEC 135.16 USB 108.22 TSS 97.01100000000001 TMO 477.8668 HCP 27.03 ZBH 123.63 WM 90.4383 TLRY 130.4 YUM 86.89 WBA 68.56 CGC 45.72 XRX 27.86 LUV 61.3 PCG 46.18 KMI 17.7 CRON 9.88 SYMC 40.32 FTI 30.63 PG 82.95 NLSN 26.0 
['0.07700000000008345 asset values 5952.944']
CRM 153.21 UDR 277.2 VRSK 120.46 ROST 97.4668 CTL 21.5 HAS 198.58 VMW 611.08 TTWO 134.53 SPGI 207.76 TGT 88.95 TSCO 454.09999999999997 WELL 132.46 NKE 79.6 V 147.8 UNH 267.6071 UAL 88.3 QCOM 69.98 O 116.74 WMT 476.8 ADM 100.16 SSTK 157.92000000000002 PFE 41.37 CMCSA 36

KeyError: 238

In [528]:
print('Final cash on hand:', cash)
for val in portfolio:
    cash += portfolio[val]*stocks[val][last_day]
print('Total asset val:',cash)

Final cash on hand: 0.07700000000008345
Total asset val: 5952.944


### Looking at Results

From the results, we see that the algorithm did pretty well. Everyday, it would check to see which stock price is trending and buy/sell/hold if necessary resulting in a final asset value of 5952.944 compared to our beginning capital of 4000. Along the way there were some data errors where some values were missing - resulting from incomplete/bad data from the Robinhood historical API. One note to point out, this data is not the adjusted close, but only the close price. Hence, some values could be a little shaky. All in all, 5952 represents a 48.8% total return. Let's see if our model could do better

### Building the Model

Given that our input is relatively small - 6 inputs to 1 output, and that the model in the paper is not very dense, in fact it is a single layer, I decided to keep the model itself not too deep but still follow the activation functions that the paper introduces. I use 3 layers with 'tanh' as the activation function, MSE as the loss function, and Adam as the optimizer.


In [490]:

def createPredictor():
    p = Sequential()
    p.add(Dense(32, input_shape = (6,), kernel_initializer = initializers.RandomNormal(stddev = 0.02)))
    p.add(Dense(64, activation = 'tanh'))
    p.add(Dense(36, activation = 'tanh'))
    p.add(Dense(1, activation = 'tanh'))
    p.compile(loss='mean_squared_error', optimizer = adam, metrics = ['accuracy'])
    p.summary()
    return p

### Training Data

Since the model only takes in 6 inputs and spits out 1 output, with around 505 stocks and 252 days of trading that gives us a total of 127260 entries of data. However, taking out NaN values and incomplete data brings it down to just 118208 entries. The data points are then randomly shuffled, and then split into ~80% for training and ~20% for testing. The data is also scaled between (-1, 1) for the model to learn easier on.

In [491]:
#def get_good_data(stocks15, macd, K, D, RSI, R, outputTrend):
input_arr = []
output_arr = []
for i in range(len(stocks15)):
    if i % 50 == 0:
        print(i)
    for j in stocks_list:
        if math.isnan(stocks15[j][i]) or math.isnan(Ks[j][i]) or math.isnan(D[j][i]) or math.isnan(RSI[j][i]) or math.isnan(R[j][i]) or math.isnan(outputTrend[j][i]):
            continue
        else:
            output_arr.append(outputTrend[j][i])
            input_arr.append([stocks15[j][i], macd[j][i], Ks[j][i], D[j][i], RSI[j][i], R[j][i]])

0
50
100
150
200
250


In [492]:
#shuffling data
input_arr = np.array(input_arr)
output_arr = np.array(output_arr)
indices = np.arange(input_arr.shape[0])
np.random.shuffle(indices)
input_arr = input_arr[indices]
output_arr = output_arr[indices]

#putting into dataframes to scale between -1, 1
input_arr = pd.DataFrame(input_arr)
output_arr = pd.DataFrame(output_arr)
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
input_arr = pd.DataFrame(scaler.fit_transform(input_arr))
scaler_out = preprocessing.MinMaxScaler(feature_range=(-1,1))
output_arr = pd.DataFrame(scaler_out.fit_transform(output_arr))

In [495]:
input_train = input_arr.values[:94000]
input_test = input_arr.values[94000:]
output_train = output_arr.values[:94000]
output_test = output_arr.values[94000:]

In [496]:
predictor = createPredictor()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                224       
_________________________________________________________________
dense_10 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_11 (Dense)             (None, 36)                2340      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 37        
Total params: 4,713
Trainable params: 4,713
Non-trainable params: 0
_________________________________________________________________


In [497]:
predictor.fit(input_train, output_train, 128, 10)

Epoch 1/10
94000/94000 [==============================] - 3s 27us/step - loss: 0.3055 - acc: 0.3918
Epoch 2/10
94000/94000 [==============================] - 2s 19us/step - loss: 0.2911 - acc: 0.3946
Epoch 3/10
94000/94000 [==============================] - 2s 19us/step - loss: 0.2905 - acc: 0.3943
Epoch 4/10
94000/94000 [==============================] - 2s 21us/step - loss: 0.2902 - acc: 0.3946
Epoch 5/10
94000/94000 [==============================] - 2s 25us/step - loss: 0.2894 - acc: 0.3949
Epoch 6/10
94000/94000 [==============================] - 2s 20us/step - loss: 0.2858 - acc: 0.3965
Epoch 7/10
94000/94000 [==============================] - 2s 22us/step - loss: 0.2804 - acc: 0.4015
Epoch 8/10
94000/94000 [==============================] - 2s 26us/step - loss: 0.2786 - acc: 0.4043
Epoch 9/10
94000/94000 [==============================] - 2s 21us/step - loss: 0.2779 - acc: 0.4060
Epoch 10/10
94000/94000 [==============================] - 2s 21us/step - loss: 0.2776 - acc: 0.4078

In [498]:
predictor.evaluate(input_test, output_test)

24208/24208 [==============================] - 1s 37us/step


[0.27804048336741943, 0.4054444811632518]

### Evaluating the Model

At the end of our short 10 epoch training, our model was at a loss of 0.277 and accuracy of 0.408 on training data and at a loss of 0.278 and accuracy of 0.405 on testing data. Now to see how it translates to the historical data once again. One thing to note is the model has trained on around ~80% of these data points even though they have been scrambled.

In [524]:
#testing model outputs
holdings = 0
cash = 4000
portfolio = {}
stocks_list = list(outputTrend)
logger = {}
# i running for number of trading days
last_day = 0
for i in range(20, len(outputTrend)-4):
    logs = []
    buys = []
    sells = []
    hold = []
    
    #gather data for prediction
    input_data = []
    input_header = []
    for j in stocks_list:
        if math.isnan(stocks15[j][i]) or math.isnan(Ks[j][i]) or math.isnan(D[j][i]) or math.isnan(RSI[j][i]) or math.isnan(R[j][i]) or math.isnan(outputTrend[j][i]):
            continue
        else:
            input_header.append(j)
            input_data.append([stocks15[j][i], macd[j][i], Ks[j][i], D[j][i], RSI[j][i], R[j][i]])    
    input_data = pd.DataFrame(input_data)
   # print(input_data)
    input_data_normal = pd.DataFrame(scaler.transform(input_data))
    
    pred_normal = predictor.predict(input_data_normal)
    
    pred_normal = pd.DataFrame(pred_normal)
    pred = pd.DataFrame(scaler_out.inverse_transform(pred_normal))
    out = pred.values
    out.shape = (1, len(pred))
    pred = pd.DataFrame(out)
    pred.columns = input_header
    
    temp = cash
    assets = ''
    for asset in portfolio:
        temp += portfolio[asset]*stocks[asset][i]
        assets = assets + asset + ' ' + str(portfolio[asset]*stocks[asset][i])+" "
    print(assets)
    logs.append(str(cash)+' asset values '+str(temp))
    
    # j for each stock
    for j in stocks_list:
        if j not in pred:
            continue
        if pred[j][0] > 0.5 and allTrends[j][i] == 1:
            buys.append((j, outputTrend[j][i]))
        elif pred[j][0] < 0.5 and allTrends[j][i] == -1:
            sells.append(j)
        else:
            hold.append(j)
    sorted_buys = sorted(buys, key = lambda x: x[1])[::-1]
    
    # here is to get the current price
    if len(sorted_buys) > 10:
        sorted_buys = sorted_buys[:20]

    for sell in sells:
        if sell in portfolio:
            #SELL THE STOCK
            #cash goes up by how much it is now
            cash += portfolio[sell]*stocks[sell][i]
            logs.append('sold '+sell+' '+str(portfolio[sell])+' @'+str(stocks[sell][i]))
            del portfolio[sell]
    #all sells have been sold
    cash_per_stock = cash/len(sorted_buys)
    buyCount = 0
#     print(sorted_buys)
    for buy in sorted_buys:
        if stocks[buy[0]][i] >= cash_per_stock and cash-stocks[buy[0]][i] >= 0:
            cash -= stocks[buy[0]][i]
            buyCount += 1
            cash_per_stock = cash/(len(sorted_buys)-buyCount)
            #bought 1 stock
            if buy[0] in portfolio:
                portfolio[buy[0]] += 1
            else:
                portfolio[buy[0]] = 1
            logs.append('buy '+buy[0]+' 1 @'+str(stocks[buy[0]][i]))
        elif stocks[buy[0]][i] < cash_per_stock:
            num_shares = int(cash_per_stock/stocks[buy[0]][i])
            cash -= stocks[buy[0]][i] * num_shares
            logs.append('buy '+buy[0]+' '+str(num_shares)+' @'+str(stocks[buy[0]][i]))
            #bought however many < cash_per_stock
            if buy[0] in portfolio:
                portfolio[buy[0]] += num_shares
            else:
                portfolio[buy[0]] = num_shares
    logger[i] = logs
    last_day = i
    print(logs)
            
print('Final cash on hand:', cash)
for val in portfolio:
    cash += portfolio[val]*stocks[val][last_day]
print('Total asset val:',cash)


['4000 asset values 4000', 'buy SSTK 5 @34.82', 'buy BOX 9 @21.53', 'buy ZTS 3 @65.5382', 'buy YUM 2 @75.2048', 'buy WY 5 @34.3524', 'buy WAT 1 @188.14', 'buy VRSN 1 @108.94', 'buy URI 1 @144.4', 'buy TSN 2 @70.3533', 'buy FOX 7 @26.603', 'buy FOXA 7 @27.26', 'buy SYK 1 @149.1751', 'buy CRM 2 @98.9', 'buy RHI 3 @50.7666', 'buy PX 1 @140.9059', 'buy PYPL 2 @70.97', 'buy NWS 14 @13.9089', 'buy MSCI 1 @120.6207', 'buy KORS 4 @49.95', 'buy MCD 1 @162.2256']
SSTK 172.85 BOX 185.4 ZTS 194.64690000000002 YUM 149.1852 WY 169.52249999999998 WAT 187.55 VRSN 107.96 URI 141.48 TSN 140.6672 FOX 180.4621 FOXA 185.1997 SYK 148.6101 CRM 196.88 RHI 149.9115 PX 140.2168 PYPL 139.6 NWS 193.3442 MSCI 120.3241 KORS 197.76 MCD 159.3381 
['694.6245000000001 asset values 3955.5329', 'buy ZTS 1 @64.8823', 'buy WLTW 1 @164.02', 'buy WDC 1 @86.4097', 'buy WMT 1 @86.6011', 'buy UDR 1 @37.5663', 'buy TSN 1 @70.3336', 'buy TRV 1 @131.4718', 'buy STX 1 @37.4602', 'buy NWS 1 @13.8103']
SSTK 173.70000000000002 BOX 18

['9.180300000000074 asset values 4227.2949']
SSTK 201.25 ZTS 352.33400000000006 YUM 233.6946 WY 176.63049999999998 WAT 198.21 VRSN 113.75 URI 150.09 TSN 230.02979999999997 FOX 211.47629999999998 FOXA 216.1474 CRM 215.16 RHI 161.883 PX 148.2888 PYPL 152.76 NWS 255.688 MSCI 125.5294 KORS 225.16 MCD 162.6353 WMT 95.2173 UDR 38.1705 TRV 127.0422 STX 37.5269 XYL 65.854 VFC 69.1826 XEL 49.2302 DIS 101.7963 PGR 50.4783 L 48.9747 CMCSA 35.6062 
['9.180300000000074 asset values 4258.976599999999', 'sold UDR 1 @38.1705', 'buy FOX 1 @30.2109', 'buy NWS 1 @15.9805']
SSTK 202.75 ZTS 353.13 YUM 234.2595 WY 175.803 WAT 200.27 VRSN 115.18 URI 151.5 TSN 232.9506 FOX 237.088 FOXA 212.74679999999998 CRM 214.74 RHI 161.883 PX 148.5152 PYPL 152.02 NWS 267.4746 MSCI 126.1344 KORS 224.6 MCD 162.8401 WMT 95.227 TRV 126.8955 STX 37.9362 XYL 66.4191 VFC 70.2964 XEL 48.94 DIS 101.1172 PGR 50.7427 L 49.1041 CMCSA 35.2813 
['1.1594000000000726 asset values 4257.004099999998', 'sold WY 5 @35.1606', 'buy WMT 1 @95.2

SSTK 298.69 ZTS 428.5944 YUM 243.9225 URI 161.17 TSN 238.9878 FOX 271.3424 FOXA 242.0292 RHI 160.5771 PX 147.585 PYPL 148.52 NWS 275.8607 KORS 245.64 MCD 169.9049 WMT 190.7756 TRV 130.3425 STX 39.9259 XYL 66.1316 VFC 72.1271 DIS 109.6883 PGR 53.9937 L 49.2656 CMCSA 38.5209 TSCO 65.9971 RF 16.637 WYNN 162.9671 TWTR 22.58 ZBH 112.1005 TXT 54.33 VIAB 29.1241 UA 12.45 ZION 48.8356 VRSK 94.73 
['0.8840999999999894 asset values 4404.2307', 'sold TSN 3 @79.6626', 'buy OKE 1 @51.3736', 'buy NBL 1 @26.1805', 'buy MOS 1 @24.8343', 'buy COP 1 @51.6042', 'buy CF 1 @40.1646', 'buy HBI 1 @20.7428', 'buy MRO 1 @15.0369']
SSTK 301.0 ZTS 429.7884 YUM 244.6956 URI 164.29 FOX 271.3424 FOXA 242.7929 RHI 163.6572 PX 148.0402 PYPL 151.3 NWS 274.1828 KORS 246.6 MCD 170.8077 WMT 190.7364 TRV 132.6135 STX 40.001999999999995 XYL 66.9942 VFC 73.1694 DIS 110.3827 PGR 54.306999999999995 L 49.5645 CMCSA 39.1019 TSCO 67.6178 RF 16.8436 WYNN 163.9845 TWTR 22.23 ZBH 114.5051 TXT 55.1692 VIAB 29.6437 UA 13.62 ZION 49.6

SSTK 305.2 ZTS 448.6026 YUM 246.06330000000003 URI 172.43 FOX 286.884 FOXA 253.5477 RHI 167.7147 PX 161.1818 PYPL 158.38 NWS 289.27540000000005 KORS 255.28 MCD 170.2974 WMT 197.1786 TRV 130.2344 STX 44.818999999999996 XYL 68.9077 VFC 74.8608 DIS 109.0633 L 50.9393 CMCSA 40.146 TSCO 78.5581 RF 17.493 WYNN 161.3869 TWTR 24.17 ZBH 123.645 TXT 58.3962 VIAB 28.4673 UA 14.1 ZION 51.4054 VRSK 96.73 OKE 55.0265 NBL 30.9397 MOS 26.0411 COP 57.1484 CF 41.5844 HBI 21.2013 MRO 17.9508 WU 20.451 
['43.5886 asset values 4599.289699999999', 'sold UA 1 @14.1', 'sold DIS 1 @109.0633', 'buy ZTS 1 @74.7671', 'buy FOX 1 @35.8605', 'buy FOXA 1 @36.2211']
SSTK 309.60999999999996 ZTS 515.0075 YUM 245.43869999999998 URI 173.2 FOX 315.0729 FOXA 282.3144 RHI 165.2268 PX 160.9245 PYPL 158.74 NWS 287.5992 KORS 255.68 MCD 170.268 WMT 195.7644 TRV 129.8804 STX 45.4759 XYL 68.41199999999999 VFC 74.8509 L 51.019 CMCSA 40.6205 TSCO 77.6093 RF 17.7881 WYNN 160.0041 TWTR 24.25 ZBH 121.9923 TXT 58.3463 VIAB 28.585 ZION 5

['6.9553000000000615 asset values 4809.778499999999', 'sold COP 1 @57.6712', 'sold HBI 1 @21.2209', 'sold MAT 2 @15.99', 'sold MCD 1 @169.2572', 'buy TPR 1 @46.8649', 'buy RHT 1 @131.46', 'buy OMC 1 @74.9161']
ZTS 535.5364 YUM 251.5041 URI 181.11 FOX 325.5102 FOXA 292.624 PYPL 170.64 KORS 264.0 WMT 209.3758 TRV 147.3899 STX 53.3306 XYL 143.2884 VFC 79.7872 L 51.4574 CMCSA 42.044 RF 18.9196 TWTR 25.81 TXT 58.61600000000001 VIAB 65.5218 ZION 107.2162 VRSK 100.05 OKE 58.0785 WU 20.208 PBCT 19.2948 NFLX 270.3 NKTR 83.61 QRVO 143.54 SRCL 75.36 TRIP 208.02 TPR 138.2148 FE 31.8538 IPG 21.3021 HAS 93.2998 GD 220.4374 CTL 16.340999999999998 RHT 131.38 OMC 74.7698 
['33.843600000000066 asset values 4763.5852', 'buy VIAB 1 @32.7609']
ZTS 543.144 YUM 249.6906 URI 179.65 FOX 327.47220000000004 FOXA 293.496 PYPL 156.8 KORS 263.72 WMT 207.2546 TRV 147.4685 STX 52.5963 XYL 148.4044 VFC 79.2759 L 51.3976 CMCSA 41.4114 RF 19.2246 TWTR 27.14 TXT 59.7849 VIAB 97.6653 ZION 109.1014 VRSK 100.26 OKE 58.9469 

...

VRSK 212.52 NFLX 399.39 RHT 138.46 MSI 115.0034 PSA 680.5101 WY 72.3 AES 65.4165 UDR 184.75799999999998 O 107.4234 NDAQ 91.2094 CNC 123.35 APA 43.9221 APC 72.7809 TDG 339.14 OXY 81.8486 SYY 203.55509999999998 OKE 69.72 KIM 34.3248 EXR 99.9992 NKE 72.3817 VMW 437.66999999999996 WLTW 304.86 FOXA 142.46699999999998 TWTR 89.68 TRIP 113.06 VFC 81.5968 MAT 17.11 WELL 178.32 VTR 110.006 V 132.3521 UAL 71.02 FOX 47.228 WMT 85.4777 KMI 17.3523 HBI 22.1161 WMB 26.8565 VIAB 29.4926 VRTX 151.5 VZ 48.735 SO 45.5012 SLG 99.3478 RSG 68.8731 DGX 111.5444 
['12.752600000000143 asset values 5652.9324']
VRSK 212.12 NFLX 390.39 RHT 132.74 MSI 114.9337 PSA 675.1611 WY 71.26 AES 65.565 UDR 184.3615 O 107.7792 NDAQ 90.6819 CNC 122.11 APA 45.6325 APC 74.0359 TDG 338.36 OXY 82.9277 SYY 203.10750000000002 OKE 69.75 KIM 33.7348 EXR 98.7017 NKE 71.1747 VMW 426.90000000000003 WLTW 300.66 FOXA 145.8135 TWTR 87.4 TRIP 111.96 VFC 80.7709 MAT 16.78 WELL 179.73 VTR 110.6174 V 130.8244 UAL 69.57 FOX 48.1044 WMT 86.3824 

VRSK 336.03000000000003 RHT 146.22 MSI 120.8492 AES 65.02000000000001 O 108.9718 NDAQ 93.4386 CNC 133.23 APA 45.7419 TDG 361.03 SYY 211.46999999999997 NKE 77.559 WLTW 315.12 TRIP 118.7 VFC 88.3525 WELL 250.36 VTR 116.58 V 138.2533 UAL 72.26 WMT 174.25599999999997 KMI 17.5994 WMB 80.80680000000001 VRTX 362.44 VZ 51.59 SO 141.30149999999998 SLG 101.96 CTL 19.136 CRM 146.96 INFO 52.11 AIZ 214.803 MNST 61.07 DVN 44.1257 CTSH 81.7412 NEE 168.3969 JEF 22.5661 XEL 137.1291 WFC 56.5707 SCG 37.9715 PRGO 74.8498 NOV 43.801 HSY 92.0817 ES 58.5593 EIX 65.66 KO 44.3527 CINF 70.0597 WEC 129.0056 ZBH 112.63 WM 83.3445 UDR 37.25 TSS 88.8019 TGT 76.6123 SRCL 67.82 
['17.500800000000147 asset values 5834.0495']
VRSK 337.46999999999997 RHT 147.58 MSI 121.9446 AES 63.6335 O 108.5946 NDAQ 93.9959 CNC 133.91 APA 45.006 TDG 362.88 SYY 213.20999999999998 NKE 77.2797 WLTW 314.54 TRIP 120.74 VFC 88.064 WELL 248.56 VTR 115.84 V 139.4315 UAL 72.62 WMT 175.3496 KMI 17.4907 WMB 79.9764 VRTX 365.18 VZ 51.43 SO 141.1

VRSK 574.05 MSI 120.40100000000001 AES 66.704 O 111.5384 NDAQ 90.9506 CNC 132.8 TDG 368.68 SYY 201.20999999999998 NKE 77.3495 VFC 90.8399 WELL 314.8 V 138.0436 UAL 80.02 WMT 175.449 KMI 17.53 WMB 89.16929999999999 VZ 206.92 SLG 103.74 CRM 137.68 INFO 53.45 AIZ 217.4288 MNST 58.77 NEE 166.8073 JEF 23.7396 XEL 138.0816 WFC 171.6774 SCG 40.1442 NOV 47.367 HSY 95.8344 ES 59.2536 EIX 66.08 KO 45.9987 CINF 74.7376 WEC 129.9974 ZBH 125.42 WM 89.4135 UDR 38.57 TSS 91.77799999999999 TGT 78.3389 SRCL 70.62 TLRY 22.93 YUM 79.47 XRX 25.49 HBAN 30.705 TPR 46.159 M 37.5464 VRSN 296.12 WBA 66.44 UHS 121.3261 TRV 128.211 KEY 20.9218 
['4.80830000000012 asset values 5861.5109']
VRSK 573.35 MSI 121.0184 AES 67.75 O 113.3914 NDAQ 91.3089 CNC 135.1 TDG 367.67 SYY 204.12 NKE 78.4568 VFC 91.6359 WELL 315.85 V 138.7126 UAL 80.85 WMT 176.483 KMI 17.68 WMB 91.86779999999999 VZ 207.92 SLG 103.17 CRM 141.14 INFO 53.35 AIZ 216.8518 MNST 59.35 NEE 168.3472 JEF 23.5507 XEL 138.7068 WFC 173.5833 SCG 41.2006 NOV 47.3

VRSK 587.1 MSI 122.4824 O 117.2568 NDAQ 92.5927 CNC 143.71 SYY 299.32 NKE 79.5541 WELL 400.20000000000005 V 141.33 UAL 82.01 WMT 195.7 VZ 219.16 SLG 105.86 CRM 145.02 INFO 53.76 MNST 62.17 NEE 174.03 XEL 191.6492 WFC 176.57999999999998 SCG 38.3702 NOV 43.791000000000004 HSY 100.7488 ES 62.7251 CINF 75.2342 WEC 136.52 ZBH 123.86 WM 90.876 UDR 39.83 TSS 94.764 TGT 83.04 TLRY 30.69 YUM 167.62 XRX 53.88 HBAN 31.9142 TPR 51.0659 VRSN 301.5 WBA 69.99 UHS 127.7011 TRV 129.6625 SBUX 53.56 REG 65.21 PBCT 18.72 PPL 29.4838 SSTK 152.57999999999998 CGC 33.75 VLO 111.0 UPS 121.8 PGR 66.28 PAYX 72.32 USB 53.29 BSX 34.41 
['0.5438000000000756 asset values 6056.2158', 'sold SCG 1 @38.3702', 'buy NI 1 @27.6']
VRSK 588.05 MSI 124.4741 O 117.1172 NDAQ 92.7121 CNC 143.08 SYY 300.72 NKE 81.9781 WELL 399.78 V 140.94 UAL 85.22 WMT 192.0 VZ 218.6 SLG 105.5 CRM 145.71 INFO 54.03 MNST 62.09 NEE 173.1657 XEL 191.4112 WFC 177.09 NOV 44.1206 HSY 101.7019 ES 62.6061 CINF 75.6216 WEC 136.5 ZBH 124.67 WM 90.7069 UDR 

KeyError: 238

In [525]:
print('Final cash on hand:', cash)
for val in portfolio:
    cash += portfolio[val]*stocks[val][last_day]
print('Total asset val:',cash)

Final cash on hand: 6.186100000000053
Total asset val: 6211.3279


### Results and Conclusion

Again with the complete testing finishing early at day 218 of trading, some stocks do not have trend data till the end, the results still look very promising. With only 10 epochs of training, the model was able to predict accurate enough trading signals that exceeded that of the original base data, turning a return at 6211, or ~55.28% return from 4000. Thus, the results are indeed promising but further testing is necessary.